# Task II: NVE molecular dynamics simulations (Part I, Week03)

## Startup

Set up the kernel

<center><img src="figures/fig1.png" width=1100 height=240 /></center>

<center><img src="figures/fig2.png" width=350 height=240 /></center>

run the following cells using `shift` + `enter`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import typing as ty
from utilities import plot_velocity, plot_data
from MD import *

# Goals
The obejective of this exercise is 

- Step 1.1 Generate the starting atomic positions, the starting velocities
- Step 1.2 Run the MD code
- Step 1.3 Change the system temperature to the desired value 
- Step 1.4 Equilibrate the sample
- Step 1.5 (Optional) check the conservation of energy as a function of the integration step $\Delta t$;
study the dependency of the ﬂuctuations on the system size (i.e., as a function of N )

Reference:
1. Rahman, A. (1964). Correlations in the motion of atoms in liquid argon. Physical review, 136(2A), A405

## Step 1.1 Running `crystal()`

First of all, a conﬁguration has to be generated to start from. 

The function `crystal()` is used to arrange atoms in a crystalline fcc structure.

The function `crystal()` takes two arguments: 
1. the number of units fcc cells along each direction `Ncells`.
2. the lattice spacing `lat_par`.

The number of unit fcc cells (containing 4 atoms each) to stack along the three directions: choose them in order to get a cubic box with same number of particles (`N = 864`) used in [1], hence select `6` unit cells along each axis so that N will be equal to 4 × (6 × 6 × 6) = 864 (in general you should not put less unit cells than what suggested to satisfy the minimum image criterion, but `6` cells is more than enough in this example). This number of cells, combined with the lattice parameter chosen below, gives a box size approximately equal to that in [1] (L = $10.229$ in L.J. units, please see the notes), so that the densities will be the same too. The lattice spacing of the fcc crystal is the equilibrium lattice spacing of the LJ potential is $1.5496$, but here we choose a value, a = $1.7048$, that corresponds to the density studied by Rahman [1], i.e. $1.374$ $g· cm^{−3}$ for Ar (with atomic mass approx. $M = 6.69\times10^{−23}$ $g$).

The function `crystal()` returns two arrays: coordinates and velocities, the latter assigned randomly according to Gaussian distribution.

<div class="alert alert-block alert-info"><b>TODO:</b> Fill the `Ncells` and `lat_par` in the following code </div>

In [ ]:
# Step 1.1 
# Create a crystalline fcc structure
#############################################################

Ncells =            # Number of unit cells along each axis
lat_par =           # Lattice parameter
L = lat_par*Ncells  # Size of the simulation box
N = 4*Ncells**3     # Number of atoms in the simulation box

# Generate fcc structure
pos, vel = crystal(Ncells, lat_par)

# Write positions and velocities into a file
dump_pos_vel('sample10.dat', pos, vel, N, L)

Nice, now you have sucessufully generated a initial structure for launching MD simulations. Let's plot the distribution of velocities, it is supposed to be approximately a gaussian distribution

In [ ]:
plot_velocity(vel)

## Step 1.2  Running the MD code

In order to run the MD code, you need to call `run_NVE()` which takes six compulsory
arguments: 
- `coordinates`
- `velocities`
- `box size`
- `No. of steps` 
- `No. of atoms` 
- `integration step`

The following code will perform a constant energy calculation (NVE ensemble)
with `200` steps (using a time step of `0.003`), continuing from `sample10.dat` previously
generated (or created by crystal), and writing on `sample11.dat` at the end.

All the MD results are stored in `output`. The accessible keys are ['nsteps', 'pos', 'vel', 'EnPot', 'EnKin']

<div class="alert alert-block alert-info"><b>TODO:</b> Fill the `nsteps` and `dt` in the following code </div>

In [ ]:
# Step 1.2 
# Run a test simulation 
#############################################################

nsteps =        # Number of steps
dt =            # Integration step

# Read crystal shape, positions and velocities from a file
N, L, pos, vel = read_pos_vel('sample10.dat')

# Perform simulation and collect the output into a dictionary
output = run_NVE(pos, vel, L, nsteps, N, dt)

# Write positions and velocities into a file
dump_pos_vel('sample11.dat', output['pos'], output['vel'], N, L)

## Step 1.3 Compute velocities and Change to the desired temperature

In order to bring the sample close to the desired temperature (through constant velocity
rescaling), we ﬁrst need to compute the velocities for the atomic conﬁguration generated
with crystal. A small number of time steps (here, `200`) is sufﬁcient for this purpose.

For this step, we will adopt an integration time step corresponding approximately to that
used in [1] for liquid Ar ($10^{−14}$ sec., see notes for the conversion to L.J. units, approximately `0.0046`).

<div class="alert alert-block alert-info"><b>TODO:</b> Fill the `nsteps` and `dt` in the following code </div>

In [ ]:
# Step 1.3
# Compute velocities
#############################################################

nsteps = 
dt = 

# Perform simulation starting from the output of a previous run
output = run_NVE(output['pos'], output['vel'], L, nsteps, N, dt)

As an option, you can check how your results depend on the time step: the value
needed to ensure conservation of energy to a good extent depends on the temperature and on
the particle density.

Now we are ready to apply the constant velocity rescaling to our sample: at each time step
the velocities will be scaled in order to bring the instantaneous temperature of the system to
the desired value (`T = 94.4K`, which corresponds to about 0.7867 in L.J. units for Ar)

`T` is an optional argument of the function run_NVE, which default value is None. When
`T` is greater than or equal to 0, the code will run a run at constant temperature. Notice that
is NOT a constant energy dynamics, hence we are not sampling the NVE ensemble during
this run (nor the NVT ensemble, see Task3 for NVT molecular dynamics). Since we are
interested in the equilibrium properties (in the thermodynamics sense) of the system, no data
should be collected in this kind of run, however you can see how the temperature changes
during the run by plotting it against the step

<div class="alert alert-block alert-info"><b>TODO:</b> Fill the `T` in the following code </div>

In [ ]:
# Step 1.4
# Change T
#############################################################

nsteps = 200
dt = 0.0046
T =           # requested temperature

# Change T 
output = run_NVE(output['pos'], output['vel'], L, nsteps, N, dt, T)

results = {
    "nsteps": output["nsteps"],
    "Tempature": output["EnKin"]*2/3,

}

# Plot temperature vs step
plot_data(results=results)

results = {
    "nsteps": output["nsteps"],
    "EnTot": output["EnPot"] + output["EnKin"],

}
plot_data(results=results)

## Step 1.4 Equilibrate

Before starting to collect data, we need to equilibrate the sample with a short run of regular
NVE dynamics.

In [ ]:
# Equilibrate
#############################################################

nsteps = 800
dt = 0.0046

# Equilibrate
output = run_NVE(output['pos'], output['vel'], L, nsteps, N, dt)


# Write positions and velocities into a file
dump_pos_vel('sampleT94.4.dat', output['pos'], output['vel'], N, L)


results = {
    "nsteps": output["nsteps"],
    "EnTot": output["EnKin"]+output["EnPot"],
}

# Plot total energy vs step
plot_data(results)


By plotting the total energy, as a function of steps you can check that `Entot` is actually conserved (to a good approximation) in this kind of dynamics (and compare with what happens
to Etot in the constant velocity rescaling run). You can verify that the conservation of energy becomes more strict as the time step `dt` is reduced. In general, the other quantities
display much larger fluctuations, instead. Notice that the average temperature might not be
equal or not even close to the target temperature, since in the NVE dynamics is not possible to
fix this variable (sometimes this makes also difficult to compare different MD simulations).

## [Optional] Step 1.5

<div class="alert alert-block alert-info"><b>TODO:</b> check the conservation of energy as a function of the integration step dt </div>

In [ ]:
# Your simulation here

<div class="alert alert-block alert-info"><b>TODO:</b> study the dependency of the ﬂuctuations on the system size (i.e., as a function of N ) </div>

In [ ]:
# Your simulation here